# Market Basket Analysis Project

## <font color='#2F4F4F'>1. Defining the Question</font>

### a) Specifying the Data Analysis Question

Identify the top 10 products likely to be purchased together and maximise revenue.


### b) Defining the Metric for Success

This project will be considered a success when:
* we are able to confidently determine which products are likely to be purchased together
* identify fequent itemsets

### c) Understanding the Context 
Care five is a German multinational retail corporation headquartered in Berlin, Germany. It is the eighth-largest retailer in the world by revenue. It operates a chain of hypermarkets, groceries stores, and convenience stores, which as of January 2021, comprises its 12,00 stores in over 30 countries.

As a Data analyst working for one of the stores, you must perform market basket analysis to help the store maximize revenue. More specifically, your task will analyze transactional data to identify the top 10 products likely to be purchased together.


### d) Recording the Experimental Design

1. Define the business question
2. Perform data importation and loading
3. Perform data preprocessing
4. nd frequent itemsets
5. Generate association rules
6. Perform metric interpretation and provide recommendation




### e) Data Relevance

The data provided is sufficient and appropriate for answering the research question.

## <font color='#2F4F4F'>2. Data Importation & Loading</font>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max.columns', None)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [3]:
df = pd.read_csv('https://bit.ly/30A2gHO')
df.head()

,A,Quantity,Transaction,Store,Product
0,30000,2,93194,6,Magazine
1,30001,2,93194,6,Candy Bar
2,30002,2,93194,6,Candy Bar
3,30003,2,93194,6,Candy Bar
4,30004,2,93194,6,Candy Bar


In [15]:
df.sample(10)

,A,Quantity,Transaction,Store,Product
14911,44911,1,133133,1,Wrapping Paper
13206,43206,1,128375,10,Pens
6349,36349,2,110030,1,Toothbrush
4272,34272,9,105080,6,Toothbrush
1116,31116,1,96299,1,Pens
13047,43047,15,128207,8,Perfume
14607,44607,1,132305,7,Magazine
8716,38716,1,116447,2,Pens
9497,39497,14,118772,4,Greeting Cards
5366,35366,1,107729,10,Pens


In [16]:
df['A'].duplicated().sum()

0

In [ ]:
# check dataset shape
df.shape

(54808, 13)

In [9]:
# drop duplicates, if any
df.describe()

,A,Quantity,Transaction,Store
count,15001.000000,15001.000000,15001.000000,15001.000000
mean,37500.000000,2.573228,113345.603760,5.430838
std,4330.560029,5.961352,11490.761538,2.858420
min,30000.000000,1.000000,93194.000000,1.000000
25%,33750.000000,1.000000,103673.000000,3.000000
50%,37500.000000,1.000000,113237.000000,5.000000
75%,41250.000000,2.000000,122996.000000,8.000000
max,45000.000000,411.000000,133433.000000,10.000000


In [10]:
# check for missing values
df.isna().sum()

A              0
Quantity       0
Transaction    0
Store          0
Product        0
dtype: int64

In [11]:
# checking the data types
df.dtypes

A               int64
Quantity        int64
Transaction     int64
Store           int64
Product        object
dtype: object

In [14]:
# getting the unique values of each variable
cols = df.columns.to_list()

for col in cols:
    print("Variable:", col)
    print("Number of unique values:", df[col].nunique())
    print(df[col].unique())
    print()

Variable: A
Number of unique values: 15001
[30000 30001 30002 ... 44998 44999 45000]

Variable: Quantity
Number of unique values: 68
[  2   1   5   8   4  13  14  10   3   7   6  21   9  26  12  24  25  15
  16  28  18  32  17  23  62  11  27  20  19  38  22  37  45  50  56  36
  33  30  40  39  43  41  34  82  35  29  54  60  84  83  75  66  63  74
  48 200 100 411  55  87  42  47  80  70  65  77  31  44]

Variable: Transaction
Number of unique values: 6726
[ 93194  93197  93200 ... 133427 133430 133433]

Variable: Store
Number of unique values: 10
[ 6  1  8  4  7  5 10  3  2  9]

Variable: Product
Number of unique values: 17
['Magazine' 'Candy Bar' 'Pencils' 'Greeting Cards' 'Toothbrush' 'Pens'
 'Soap' 'Photo Processing' 'Perfume' 'Bow' 'Toothpaste' 'Shampoo'
 'Deodorant' 'Markers' 'Pain Reliever' 'Wrapping Paper' 'Prescription Med']



From the exploration above, we can tell there are 15000 transactions in the datatset related to 17 product types, from about 10 stores. We will now perform preprocessing

## <font color='#2F4F4F'>3. Data Preprocessing</font>

In [21]:
# distribution of products
df2 = df.groupby(["Transaction","Product"]).size().reset_index(name="Count")

df2.head()

,Transaction,Product,Count
0,93194,Candy Bar,4
1,93194,Magazine,1
2,93197,Pencils,1
3,93200,Candy Bar,3
4,93200,Magazine,1


In [26]:
txn_df = (df2.groupby(["Transaction","Product"])['Count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Transaction'))
txn_df.sample(10)

Product,Bow,Candy Bar,Deodorant,Greeting Cards,Magazine,Markers,Pain Reliever,Pencils,Pens,Perfume,Photo Processing,Prescription Med,Shampoo,Soap,Toothbrush,Toothpaste,Wrapping Paper
Transaction,,,,,,,,,,,,,,,,,
107111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113930,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
103724,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
132839,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
93698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#We then use our custom encoding function to convert 
# all the values to 0 or 1. 
# The Apriori algorithm will only take 0's or 1's.
# ---
# 

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

df3 = txn_df.applymap(encode_units)

df3.head()

Product,Bow,Candy Bar,Deodorant,Greeting Cards,Magazine,Markers,Pain Reliever,Pencils,Pens,Perfume,Photo Processing,Prescription Med,Shampoo,Soap,Toothbrush,Toothpaste,Wrapping Paper
Transaction,,,,,,,,,,,,,,,,,
93194,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
93197,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
93200,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
93206,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
93212,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [28]:
# Step 2: Generating frequent itemsets
# ---
# We'll generate the most frequent itemsets by using apriori function() 
# pass the parameters: 
# ---
# min_support = 0.01 - We set minimum-support threshold at 1%
# use_colnames = True to display the column names in our itemset columns.
# If you set use_colnames = False the itemsets will be shown in indices.
# ---
# 
df_frequent_itemsets = apriori(df3, min_support=0.01, use_colnames=True)
df_frequent_itemsets.head()

,support,itemsets
0,0.051591,(Bow)
1,0.175736,(Candy Bar)
2,0.152840,(Greeting Cards)
3,0.231936,(Magazine)
4,0.020071,(Pain Reliever)


We see that majority of the employees are in Sales and Marketing.

In [30]:
# Step 3: Generating association rules
# ---
# The final step is to generate the rules with their 
# corresponding support, confidence and lift using the 
# association_rules() function. 
# We will set the minimum threshold for lift at 1 
# and then sort the result by descending confidence value.
# Don't worry about the leverage and conviction metrics. 
# You can consider them for your further reading
# ---
#
rules = association_rules(df_frequent_itemsets, metric="lift", min_threshold=1)

# Sorting 
rules.sort_values("confidence", ascending = False, inplace = True)

# Previewing the association rules
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
45,"(Toothpaste, Pencils)",(Candy Bar),0.022748,0.175736,0.011002,0.483660,2.752198,0.007005,1.596359
22,"(Greeting Cards, Magazine)",(Candy Bar),0.037467,0.175736,0.017247,0.460317,2.619370,0.010662,1.527313
40,"(Magazine, Toothpaste)",(Candy Bar),0.029884,0.175736,0.013232,0.442786,2.519610,0.007981,1.479260
28,"(Greeting Cards, Toothpaste)",(Candy Bar),0.033304,0.175736,0.014570,0.437500,2.489530,0.008718,1.465358
21,"(Candy Bar, Magazine)",(Greeting Cards),0.039994,0.152840,0.017247,0.431227,2.821431,0.011134,1.489452
51,"(Magazine, Pencils)",(Greeting Cards),0.028546,0.152840,0.012043,0.421875,2.760244,0.007680,1.465358
50,"(Greeting Cards, Pencils)",(Magazine),0.029884,0.231936,0.012043,0.402985,1.737486,0.005112,1.286508
20,"(Candy Bar, Greeting Cards)",(Magazine),0.046090,0.231936,0.017247,0.374194,1.613350,0.006557,1.227319
57,"(Magazine, Toothpaste)",(Greeting Cards),0.029884,0.152840,0.011151,0.373134,2.441344,0.006583,1.351422
34,"(Magazine, Pencils)",(Candy Bar),0.028546,0.175736,0.010407,0.364583,2.074609,0.005391,1.297202


## <font color='#2F4F4F'>5. Summary of Findings</font>

1. Toothpaste, pencils and candybars with a lift of 2.75 indicaitng a strong association. The lift value of 2.75 means that (Toothpaste & pencil) purchase lifts the Candy bars purchase by 2.75 times.

2. The output above shows the Top 10 itemsets sorted by confidence value and all itemsets have support value over 1% and lift value over 2 majorly. 



## <font color='#2F4F4F'>6. Recommendations</font>


We can conclude that there is indeed evidence to suggest that the purchase of most items leads to the purchase of Candy bars, greeting cars & magazines. Care Five should consider bundling toothpastes,greeting cards, magazines and candy bars together as a set, the staff in the store should also be trained to cross-sell these items to customers who purchase toothpaste, pencils, greeting cards or magazines knowing that they are more likely to purchase them together, thereby increasing the store's revenue.

## <font color='#2F4F4F'>7. Challenging your Solution</font>


#### a) Did we have the right question?
Yes

#### b) Did we have the right data?
Yes, the data was sufficient to determine top selling products and item sets. 

#### c) What can be done to improve the solution?
We can incorporate transactions with more than one product types sold to further understand the associations